In [ ]:
import cv2
import numpy as np

In [29]:
# Constants
z = 0.9
m = 0.6
n = 5  # Number of Gaussian smoothing iterations
h = (1 / 16) * np.array([[1, 2, 1],
                         [2, 4, 2],
                         [1, 2, 1]], dtype=np.float32)  # Gaussian kernel

cam = cv2.VideoCapture(0)
if not cam.isOpened():
    print("Camera not detected")
    exit()


In [ ]:
while True:
    ret, frame = cam.read()
    if not ret:
        print("Failed to read frame")
        break

    # Step 1: Convert to float32
    I = frame.astype(np.float32)
    
    # Step 2: Estimate airlight by dark channel (min across channels)
    I_air = np.min(I, axis=2)  # Shape: H x W

    # Step 3: Refinement via Gaussian blur iteratively
    I_refined = I_air.copy()
    for _ in range(n):
        I_refined = cv2.filter2D(I_refined, -1, h)

    # Step 4: Enhancement using final simplified equation
    correction = 1.0 + ((1.0 / 170.0 * I_refined) ** 4) # this is the simplified equation when z = 0.9, m = 0.6

    # Step 5: Apply correction to the original image
    I_enhanced = I * correction[:, :, np.newaxis]  # Broadcast to all channels

    # Clip and convert to 8-bit
    I_enhanced = np.clip(I_enhanced, 0, 255).astype(np.uint8)

    # Display
    cv2.imshow("Input", frame)
    cv2.imshow("Enhanced", I_enhanced)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()
